In [1]:
import numpy as np
import pandas as pd
import plotly
import math
from pandas_datareader import data as wb
from portfolio_return import *


Au 21 mai 2020, les 10 entreprises suivantes représentent 50 % de l'indice :


10 Top Stocks to take :

APPLE INC.
MICROSOFT CORPORATION
AMAZON.COM, INC.
FACEBOOK
ALPHABET INC.
INTEL CORPORATION
COMCAST CORPORATION
CISCO SYSTEMS, INC.
PEPSICO, INC.
ADOBE INC.

In [72]:
#Chose inputs
#Benchmark : Nasdaq 100 -> to beat !
#Portfolio Management : 10 Higher MC companies of Nasdaq 100

Benchmark = ['NQ=F']
Nasdaq_weights = [11.018,10.209,7.894,4.138,4.093,3.95,3.828,3.754,2.233,2.152]
tickers = ['AAPL','MSFT','AMZN','GOOG','TSLA','FB','GOOGL','NVDA','PYPL','ADBE']

stock_prices_train = import_stock_prices(tickers, start='2019-09-1', end='2020-09-1')
stock_prices_test = import_stock_prices(tickers, start='2020-09-1', end='2021-09-1')

stock_benchmark_train = import_stock_prices(Benchmark, start='2019-09-1', end='2020-09-1')
stock_benchmark_test = import_stock_prices(Benchmark, start='2020-09-1', end='2021-09-1')


In [101]:


benchmark_train = compute_portfolio_metrics(stock_prices=stock_benchmark_train,tickers=Benchmark, weights=[1])
benchmark_test = compute_portfolio_metrics(stock_prices=stock_benchmark_test,tickers=Benchmark, weights=[1])

portfolios = random_strategies(stock_prices=stock_prices_train, tickers=tickers, N=5000)


KeyboardInterrupt: 

In [102]:
final_strategy

{'portfolio_returns': 1.3342786277505119,
 'portfolio_weights': array([0.03184908, 0.01380033, 0.25062282, 0.02099682, 0.27926469,
        0.02434048, 0.13670654, 0.15142518, 0.04847203, 0.04252204]),
 'portfolio_risks': 0.4267288676167632,
 'sharpe_ratios': 3.126759703889546}

In [74]:
strats_df = pd.DataFrame(portfolios)

min_risk = strats_df.iloc[strats_df['portfolio_risks'].astype(float).idxmin()]
highest_return = strats_df.iloc[strats_df['portfolio_returns'].astype(float).idxmin()]
highest_sharpe = strats_df.iloc[strats_df['sharpe_ratios'].astype(float).idxmax()]

#selectionner la meilleure stratégie :
validation_weights = highest_sharpe['portfolio_weights']
final_strategy = compute_portfolio_metrics(stock_prices=stock_prices_train, tickers=tickers,weights=validation_weights)

#Compute its metrics
validation_portfolio = compute_portfolio_metrics(stock_prices=stock_prices_test,tickers=tickers, weights=final_strategy['portfolio_weights'])

In [75]:
#Visualisation
import plotly.express as px
df = strats_df[['portfolio_returns','portfolio_risks','sharpe_ratios']]
fig = px.scatter(df, x="portfolio_risks", y="portfolio_returns", color='sharpe_ratios')
fig.show()

In [76]:
benchmark_returns = cumulative_perc(simple_return(stock_benchmark_train))
benchmark_returns

,NQ=F
Date,
2019-09-03,NaN
2019-09-04,0.013817
2019-09-05,0.031244
2019-09-06,0.031539
2019-09-09,0.027995
...,...
2020-08-26,0.574171
2020-08-27,0.569117
2020-08-28,0.574237


In [77]:
cum_perc = cumulative_perc(simple_return(stock_prices_test))*final_strategy['portfolio_weights']
cum_perc = cum_perc.sum(axis=1).to_frame()
cum_perc.columns = ['Optimal Portfolio Return']


benchmark_returns = cumulative_perc(simple_return(stock_benchmark_test))
benchmark_returns.columns = ['Benchmark']

benchmark_returns.name = 'benchmark'
cum_perc.name = 'portfolio optimal'

In [103]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cum_perc.index,
                         y=cum_perc['Optimal Portfolio Return'].values,
                         name="Optimal Portfolio",
                         line_shape='linear'
                         ))
fig.add_trace(go.Scatter(x=benchmark_returns.index, y=benchmark_returns['Benchmark'].values,
                         name="Benchmark (NSDQ 100)",
                         line_shape='linear'))
fig.update_layout(
    autosize=False,
    width=1400,
    height=800,)

fig.show()

In [98]:
(cum_perc['Optimal Portfolio Return'].fillna(method="ffill").tail(125).values - benchmark_returns['Benchmark'].fillna(method="ffill").tail(125).values).mean()

0.08303977162073777

In [79]:
validation_portfolio

{'portfolio_returns': 0.4170285194885487,
 'portfolio_weights': array([0.03184908, 0.01380033, 0.25062282, 0.02099682, 0.27926469,
        0.02434048, 0.13670654, 0.15142518, 0.04847203, 0.04252204]),
 'portfolio_risks': 0.32153455671753034,
 'sharpe_ratios': 1.2969944000604272}

In [80]:

benchmark_train = compute_portfolio_metrics(stock_prices=stock_benchmark_train,tickers=Benchmark, weights=[1])
benchmark_test = compute_portfolio_metrics(stock_prices=stock_benchmark_test,tickers=Benchmark, weights=[1])

benchmark_test

{'portfolio_returns': simple_return_NQ=F    0.260366
 dtype: float64,
 'portfolio_weights': [1],
 'portfolio_risks': simple_return_NQ=F    0.214596
 dtype: float64,
 'sharpe_ratios': simple_return_NQ=F    1.213287
 dtype: float64}

In [81]:
validation_portfolio

{'portfolio_returns': 0.4170285194885487,
 'portfolio_weights': array([0.03184908, 0.01380033, 0.25062282, 0.02099682, 0.27926469,
        0.02434048, 0.13670654, 0.15142518, 0.04847203, 0.04252204]),
 'portfolio_risks': 0.32153455671753034,
 'sharpe_ratios': 1.2969944000604272}